# SPARCED's Interactive Tutorial

Welcome to SPARCED!

This tutorial covers the internals of SPARCED's pipeline, including compilation and simulation. It aims to explain:
  - which steps are followed to compile and run the model
  - which arguments are required
  - how those arguments are processed

## (Optional) Place yourself into the correct folder

SPARCED is designed to be run from the ```SPARCED/src``` subfolder. If you want to run SPARCED from elsewhere (which is the case of this tutorial, that is located by default in the ```examples/notebooks``` subfolder), you will first need to add a path to that folder.

**WARNING:** If you moved this tutorial from its default location, please make sure the relative path is still correct.

In [1]:
import os
import sys

from pathlib import Path
import yaml

sys.path.insert(0, os.path.abspath("../../SPARCED/src"))

## 0 - Imports

It is a good practice to add the imports at the beginning of your script or notebook.

In [2]:
# Step 1: model creation
from compilation.launcher import load_compilation_input_files
from compilation.create_model import create_model
# Step 2: model simulation
from simulation.launchers import load_simulation_input_files
from simulation.run_model import run_experiment

## 1 - Create the Model: Compilation

### Overview

The model creation pipeline works as follow:
  1. Some TSV (tab separated values) **input files** are read to generate an Antimony model.
  2. Next, this **Antimony model** is converted into SBML format.
  3. Finally, the **SBML model** is compiled into an AMICI model
SPARCED can only run if an **AMICI model** was generated.

_NB: Although it is possible to run SPARCED starting from an Antimony or an SBML file, in this tutorial will only cover how to create a SPARCED model from the input files._

### Details

Instead of using the launcher, we will cover what it does by directly calling the ```create_model()``` function.

In [3]:
# Model name
# The launcher gets and sanitizes that variable based on the "name" argument
model_name = "SPARCED_tutorial"
# Model path
# The launcher automatically reconstructs that path based on:
#     - the "model" argument, which is the path to the models directory
#     - the model_name variable above
model_path = Path("../../SPARCED/models/SPARCED_tutorial")
# Input data files
# The launcher loads a dictionnary containing the paths of all the input data files based on:
#     - the model_path variable name above
#     - the "input_data" argument, which is the name of the folder containing this data
#     - the "yaml" argument, which is the name of the YAML file containing description of the data organization
input_data_folder = Path("../../SPARCED/models/SPARCED_tutorial/data")
input_files = load_compilation_input_files(input_data_folder, "config.yaml")
# Output parameters
# The launcher automatically reconstructs that path based on:
#     - the model_path variable name above
#     - the "output_parameters" argument, which is the desired name for this file
output_parameters_path = Path("../../SPARCED/models/SPARCED_tutorial/out_Parameters.txt")
# Runtime booleans
# The launcher gets those variables based on the "wild" and "verbose" arguments
is_SPARCED = True
verbose = True
create_model(model_name, model_path, input_files, output_parameters_path, verbose, is_SPARCED)
# NB: AMICI is very verbose, so don't be afraid if it loads tons of lines.
# The model takes several minutes to compile.

Subfolder name: ant_SPARCED_tutorial.txt.
SPARCED_tutorial: Success loading Antimony file
Subfolder name: sbml_SPARCED_tutorial.xml.
SPARCED_tutorial: Success converting Antimony file to SBML
Subfolder name: amici_SPARCED_tutorial.


2024-05-22 18:18:35.761 - amici.sbml_import - INFO - Finished gathering local SBML symbols        + (8.60E+00s)
2024-05-22 18:18:35.989 - amici.sbml_import - INFO - Finished processing SBML parameters          + (2.19E-01s)
2024-05-22 18:18:36.001 - amici.sbml_import - INFO - Finished processing SBML compartments        + (6.53E-04s)
2024-05-22 18:18:37.619 - amici.sbml_import - INFO - Finished processing SBML species initials   ++ (1.52E+00s)
2024-05-22 18:18:37.626 - amici.sbml_import - INFO - Finished processing SBML rate rules         ++ (2.92E-05s)
2024-05-22 18:18:37.627 - amici.sbml_import - INFO - Finished processing SBML species             + (1.62E+00s)
2024-05-22 18:18:38.955 - amici.sbml_import - INFO - Finished processing SBML reactions           + (1.32E+00s)
2024-05-22 18:18:40.619 - amici.sbml_import - INFO - Finished processing SBML rules               + (1.65E+00s)
2024-05-22 18:18:40.626 - amici.sbml_import - INFO - Finished processing SBML initial assignments + (5.0

running build_ext
Changed extra_compile_args for unix to ['-fopenmp', '-std=c++14']
building 'SPARCED_tutorial._SPARCED_tutorial' extension
Testing SWIG executable swig4.0... SUCCEEDED.
swigging swig/SPARCED_tutorial.i to swig/SPARCED_tutorial_wrap.cpp
swig4.0 -python -c++ -modern -outdir SPARCED_tutorial -I/home/chocolatecharlie/anaconda3/envs/sparced/lib/python3.12/site-packages/amici/swig -I/home/chocolatecharlie/anaconda3/envs/sparced/lib/python3.12/site-packages/amici/include -o swig/SPARCED_tutorial_wrap.cpp swig/SPARCED_tutorial.i
Deprecated command line option: -modern. This option is now always on.
creating build
creating build/temp.linux-x86_64-cpython-312
creating build/temp.linux-x86_64-cpython-312/swig
gcc -pthread -B /home/chocolatecharlie/anaconda3/envs/sparced/compiler_compat -fno-strict-overflow -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /home/chocolatecharlie/anaconda3/envs/sparced/include -fPIC -O2 -isystem /home/chocolatecharlie/anaconda3/envs/sparced/include -fPIC -I/ho

Although this might seem overwhelming, note that you may simply call the ```launch_model_creation()``` function from your script or the command line, which will collect all the default values and arguments you pass, process these and call the ```create_model()``` function for you.

A call to ```launch_model_creation()``` looks as simple as this:

In [4]:
# WARNING: the Notebook doesn"t handle well command-line arguments
# We encourage you to see the __main__.py file located in SPARCED/src for a working example
# from compilation.launcher import launch_model_creation()
# launch_model_creation()

## 2 - Run the Model: Simulation

### Overview

The model simulation pipeline works as follow:
  1. The experiment's model and **initial conditions** are loaded.
  2. A **single-cell simulation** is run until reaching the desired number of cells.
  3. Files are **saved** at the end of each single-cell simulation.

### Details

Instead of using the launcher, we will cover what it does by directly calling the ```run_experiment()``` function.
The ```run_experiment()``` function is a wrapper that will call the ```run_single_simulation()``` function for each cell in your population.

_Please note that in the example bellow, we redefined some variables previously set to give you an overview of what you need to call the ```run_experiment()``` function. In practice, you wouldn't want to redefine variables in your script of course._

In [7]:
# Model name
# The launcher gets and sanitizes that variable based on the "name" argument
model_name = "SPARCED_tutorial"
# Model path
# The launcher automatically reconstructs that path based on:
#     - the "model" argument, which is the path to the models directory
#     - the model_name variable above
model_path = Path("../../SPARCED/models/SPARCED_tutorial")
# Simulation name
# The launcher gets that variable based on the "simulation" argument
simulation_name = "Tutorial"
# Results path
# The launcher automatically reconstructs that path based on the "results" argument,
# which is the path to the directory where you wish the results to be saved
results_path = "../../SPARCED/results/New-Simulation"
# AMICI and SBML paths
# The launcher automatically reconstructs those paths based on the model_name and model_path variables above.
amici_path = Path("../../SPARCED/models/SPARCED_tutorial/amici_SPARCED_tutorial")
sbml_path = Path("../../SPARCED/models/SPARCED_tutorial/sbml_SPARCED_tutorial.xml")
# Input data files
# The launcher loads a dictionnary containing the paths of all the input data files based on:
#     - the model_path variable name above
#     - the "input_data" argument, which is the name of the folder containing this data
#     - the "yaml" argument, which is the name of the YAML file containing description of the data organization
#     - the "perturbations" argument, which is the name of the file to override default perturbations
input_data_folder = Path("../../SPARCED/models/SPARCED_tutorial/data")
input_files, perturbations = load_simulation_input_files(input_data_folder, "config.yaml", None)
# Runtime variables
# The launcher gets those variables based on the "deterministic", "population_size", "time", "exchange", "verbose" and "wild" arguments.
deterministic = True
population_size = 1
time = 1.0 # in hours
exchange = 30 # don't touch this if you don't know how a systems biology model works
verbose = True
is_SPARCED = True
run_experiment(model_name, simulation_name, results_path, amici_path, sbml_path, input_files, deterministic, population_size, \
               time, exchange, verbose, is_SPARCED, perturbations)
# It will take several minutes for the model to run.

SPARCED_tutorial: Successfully loaded <module 'SPARCED_tutorial' from '/home/chocolatecharlie/Documents/SPARCED/SPARCED/models/SPARCED_tutorial/amici_SPARCED_tutorial/SPARCED_tutorial/__init__.py'>

Tutorial n°0: Now ready to run

Tutorial n°0: Simulation is over. Now saving results,                 please do not exit.

Tutorial n°0: Simulation saved.



Although this might seem overwhelming, note that you may simply call the ```launch_experiment_simulation()``` function from your script or the command line, which will collect all the default values and arguments you pass, process these and call the ```run_experiment()``` function for you.

A call to ```launch_experiment_simulation()``` looks as simple as this:

In [8]:
# WARNING: the Notebook doesn"t handle well command-line arguments
# We encourage you to see the __main__.py file located in SPARCED/src for a working example
# from simulation.launchers import launch_experiment_simulation()
# launch_experiment_simulation()